In [18]:
# Initial imports.
import pandas as pd
import numpy as np
import psycopg2
from datetime import date, datetime, timedelta
from matplotlib import pyplot as plt
# from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree

In [2]:
# Establish connection to AWS
connection = psycopg2.connect(
    host = 'group-eight-db.cchjt3hyglcz.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'root',
    password = 'group_8_drought_2021',
    database='group_8_db_1'
    )
cursor=connection.cursor()

In [3]:
# Load weather csv files from AWS
sql = """
SELECT *
FROM "drought_time_series"
"""
weather_df = pd.read_sql(sql, con=connection)
weather_df.head()

In [4]:
# Load soil csv files from AWS
sql = """
SELECT *
FROM "soil_data"
"""
soil_df = pd.read_sql(sql, con=connection)
soil_df.head()

In [31]:
# Loading weather data from local drive
#weather_df = pd.read_csv("data/test_timeseries.csv")
#weather_df.head()

In [32]:
# Loading soil data from local drive
#soil_df = pd.read_csv("data/soil_data.csv")
#soil_df.head()

In [7]:
# Limit the weather data to Texas
texas_weather = weather_df.loc[weather_df["fips"] >= 48000]
texas_weather = texas_weather.loc[weather_df["fips"] <= 48999]

In [8]:
# Merge the Texas weather dataframe with the soil dataframe for each county
texas_drought = pd.merge(texas_weather, soil_df, on='fips')
texas_drought

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,48001,2019-01-01,0.13,100.79,5.50,6.65,5.21,5.11,10.88,2.71,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
1,48001,2019-01-02,41.48,100.66,5.36,5.57,4.85,4.82,6.63,4.40,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
2,48001,2019-01-03,11.94,100.13,5.19,5.10,4.30,4.25,7.71,0.55,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
3,48001,2019-01-04,0.00,100.20,5.02,5.91,3.84,3.55,12.91,-0.25,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
4,48001,2019-01-05,0.00,100.56,6.34,9.13,7.25,7.09,17.84,3.22,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185669,48507,2020-12-27,0.15,98.98,5.73,13.42,5.62,9.52,23.64,5.57,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
185670,48507,2020-12-28,0.85,99.17,10.15,18.05,14.13,16.09,26.02,10.24,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
185671,48507,2020-12-29,1.56,98.71,10.87,20.17,15.15,17.66,26.36,16.61,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3
185672,48507,2020-12-30,8.47,98.39,9.59,16.53,13.12,14.83,24.90,8.39,...,0.000000,8.432994,8.432994,1,1,2,2,1,1,3


In [9]:
# Remove unnecessary data
# Drop redundant weather data
texas_drought = texas_drought.drop(['T2MDEW', 'T2MWET', 'T2M_RANGE', 'WS10M_RANGE', 'WS50M_RANGE'], axis=1)
# Drop unnecessary soil data
texas_drought = texas_drought.drop(['lat', 'lon'], axis=1)

In [10]:
# Address NaN scores
# Drop NaN drought scores
#texas_drought.dropna(inplace=True)
# Set NaN drought scores to the most recent
texas_drought.fillna(method='ffill', inplace=True)
texas_drought.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,48001,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
1,48001,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
2,48001,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
3,48001,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1
4,48001,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,50.254311,0.180426,50.434738,3,2,1,1,1,1,1


In [11]:
# Add scores for one week out to texas_drought dataframe (These are the scores we are trying to predict)
# Create new dataframe with dates and scores
week_later = texas_drought[['fips', 'date', 'score']]
week_later['prediction_date'] = week_later['date']

# Adjust the date forward seven days to get the week_before scores
time_delta = timedelta(-7)
texas_drought['date'] = pd.to_datetime(texas_drought['date'])
week_later['date'] = pd.to_datetime(week_later['date'])
week_later['prediction_date'] = week_later['date']
week_later['date'] = week_later['date'] + time_delta

# Merge the week_before scores into the texas_drought dataframe
texas_drought = pd.merge(texas_drought, week_later,  how='inner', left_on=['fips','date'], right_on = ['fips','date'])
texas_drought.head()

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a co

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,score_y,prediction_date
0,48001,2019-01-01,0.13,100.79,5.50,6.65,10.88,2.71,6.66,4.22,...,50.434738,3,2,1,1,1,1,1,0.0,2019-01-31
1,48001,2019-01-02,41.48,100.66,5.36,5.57,6.63,4.40,5.58,3.69,...,50.434738,3,2,1,1,1,1,1,0.0,2019-02-01
2,48001,2019-01-03,11.94,100.13,5.19,5.10,7.71,0.55,5.02,3.17,...,50.434738,3,2,1,1,1,1,1,0.0,2019-02-02
3,48001,2019-01-04,0.00,100.20,5.02,5.91,12.91,-0.25,4.85,3.11,...,50.434738,3,2,1,1,1,1,1,0.0,2019-02-03
4,48001,2019-01-05,0.00,100.56,6.34,9.13,17.84,3.22,7.96,2.88,...,50.434738,3,2,1,1,1,1,1,0.0,2019-02-04


In [12]:
# Address seasonality
# Add values from one year prior to texas_drought dataframe
# Create new dataframe with dates and scores
year_before = texas_drought[['fips', 'date', 'score_x']]

# Adjust the date forward seven days to get the week_before scores
# This is 51 weeks before the current date, which is 52 weeks before the predicted date, 364-x
time_delta = timedelta(+357)
year_before['date'] = pd.to_datetime(year_before['date'])
year_before['date'] = year_before['date'] + time_delta

# Merge the year_before scores into the texas_drought dataframe
texas_drought = pd.merge(texas_drought, year_before,  how='inner', left_on=['fips','date'], right_on = ['fips','date'])
texas_drought.head()

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,score_y,prediction_date,score_x_y
0,48001,2019-12-01,0.00,100.47,3.34,9.48,15.27,5.10,9.44,4.99,...,3,2,1,1,1,1,1,2.9912,2019-12-31,0.0
1,48001,2019-12-02,0.01,100.94,3.66,7.68,14.75,2.61,7.13,3.19,...,3,2,1,1,1,1,1,2.9912,2020-01-01,0.0
2,48001,2019-12-03,0.00,100.19,4.44,10.05,20.16,2.49,9.24,3.42,...,3,2,1,1,1,1,1,2.9912,2020-01-02,0.0
3,48001,2019-12-04,0.00,100.32,5.21,12.23,21.22,4.35,11.81,2.44,...,3,2,1,1,1,1,1,2.9912,2020-01-03,0.0
4,48001,2019-12-05,0.00,100.13,7.57,15.21,22.75,7.70,15.05,4.55,...,3,2,1,1,1,1,1,2.9912,2020-01-04,0.0


In [13]:
# Clean up the 'score' columns
# Reorder the columns
texas_drought = texas_drought[['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2M_MAX', 'T2M_MIN',
       'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS50M', 'WS50M_MAX',
       'WS50M_MIN', 'elevation', 'slope1', 'slope2', 'slope3',
       'slope4', 'slope5', 'slope6', 'slope7', 'slope8', 'aspectN', 'aspectE',
       'aspectS', 'aspectW', 'aspectUnknown', 'WAT_LAND', 'NVG_LAND',
       'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND',
       'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7', 'score_x_x', 'prediction_date', 'score_y',
       'score_x_y']]
# Rename the score columns
texas_drought['score_to_be_predicted'] = texas_drought['score_y']
texas_drought['current_score'] = texas_drought['score_x_x']
texas_drought['score_year_before'] = texas_drought['score_x_y']
# Drop the old score columns
texas_drought.drop(['score_x_x', 'score_y', 'score_x_y'], axis=1, inplace=True)

In [14]:
texas_drought.head()

,fips,date,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,...,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,prediction_date,score_to_be_predicted,current_score,score_year_before
0,48001,2019-12-01,0.00,100.47,3.34,9.48,15.27,5.10,9.44,4.99,...,2,1,1,1,1,1,2019-12-31,2.9912,1.9691,0.0
1,48001,2019-12-02,0.01,100.94,3.66,7.68,14.75,2.61,7.13,3.19,...,2,1,1,1,1,1,2020-01-01,2.9912,1.9691,0.0
2,48001,2019-12-03,0.00,100.19,4.44,10.05,20.16,2.49,9.24,3.42,...,2,1,1,1,1,1,2020-01-02,2.9912,2.0000,0.0
3,48001,2019-12-04,0.00,100.32,5.21,12.23,21.22,4.35,11.81,2.44,...,2,1,1,1,1,1,2020-01-03,2.9912,2.0000,0.0
4,48001,2019-12-05,0.00,100.13,7.57,15.21,22.75,7.70,15.05,4.55,...,2,1,1,1,1,1,2020-01-04,2.9912,2.0000,0.0


In [25]:
# Split the training and testing data
train_enddate = datetime(year=2016, month=12, day=31)
Train = texas_drought.loc[texas_drought["date"] <= train_enddate]
Test = texas_drought.loc[texas_drought["date"] > train_enddate]

In [26]:
# Define the training features set.
X_train = Train.copy()
X_train = X_train.drop("score_to_be_predicted", axis=1)
X_train = X_train.drop("date", axis=1)
X_train = X_train.drop("prediction_date", axis=1)
X_train = X_train.drop("fips", axis=1)
X_train

,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,WS10M_MAX,WS10M_MIN,...,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,current_score,score_year_before


In [27]:
# Define the testing features set.
X_test = Test.copy()
X_test = X_test.drop("score_to_be_predicted", axis=1)
X_test = X_test.drop("date", axis=1)
X_test = X_test.drop("prediction_date", axis=1)
X_test = X_test.drop("fips", axis=1)
X_test

,PRECTOT,PS,QV2M,T2M,T2M_MAX,T2M_MIN,TS,WS10M,WS10M_MAX,WS10M_MIN,...,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,current_score,score_year_before
0,0.00,100.47,3.34,9.48,15.27,5.10,9.44,4.99,7.25,3.42,...,50.434738,3,2,1,1,1,1,1,1.9691,0.0000
1,0.01,100.94,3.66,7.68,14.75,2.61,7.13,3.19,5.17,1.53,...,50.434738,3,2,1,1,1,1,1,1.9691,0.0000
2,0.00,100.19,4.44,10.05,20.16,2.49,9.24,3.42,6.11,1.66,...,50.434738,3,2,1,1,1,1,1,2.0000,0.0000
3,0.00,100.32,5.21,12.23,21.22,4.35,11.81,2.44,3.20,1.38,...,50.434738,3,2,1,1,1,1,1,2.0000,0.0000
4,0.00,100.13,7.57,15.21,22.75,7.70,15.05,4.55,6.56,2.71,...,50.434738,3,2,1,1,1,1,1,2.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93213,4.76,98.88,11.97,21.42,28.38,15.33,21.60,3.53,7.69,1.90,...,8.432994,1,1,2,2,1,1,3,4.9233,3.2349
93214,22.01,99.15,8.78,14.64,17.90,11.21,14.90,4.98,6.32,3.53,...,8.432994,1,1,2,2,1,1,3,4.9233,3.2349
93215,0.05,99.54,5.50,12.26,21.18,6.57,12.61,4.73,7.05,3.06,...,8.432994,1,1,2,2,1,1,3,4.9233,3.2349
93216,0.00,100.33,2.91,9.91,16.17,5.61,8.96,3.00,5.47,0.31,...,8.432994,1,1,2,2,1,1,3,4.9233,3.2349


In [28]:
# Define the training target
y_train = Train["score_to_be_predicted"].ravel()
y_train[:5]

array([], dtype=float64)

In [29]:
# Define the testing target
y_test = Test["score_to_be_predicted"].ravel()
y_test[:5]

array([2.9912, 2.9912, 2.9912, 2.9912, 2.9912])

In [187]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [188]:
# Create a random forest Regressor.
rf_model = RandomForestRegressor(n_estimators=128, max_depth=25, random_state=1) 

In [189]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [190]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [191]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

Mean Absolute Error (MAE): 0.21093419709423947
Mean Squared Error (MSE): 0.1054075453231857
Root Mean Squared Error (RMSE): 0.3246652819800505
Mean Absolute Percentage Error (MAPE): nan
Accuracy: nan


/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [159]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([2.27884367e-03, 7.97445979e-03, 3.32751585e-03, 6.95967148e-03,
       1.78307055e-03, 3.01532001e-03, 2.43951352e-03, 3.12657782e-03,
       1.03061001e-03, 1.48406655e-03, 1.27731128e-03, 1.04250508e-03,
       1.68090479e-03, 1.24366848e-03, 1.99162540e-03, 1.00899824e-03,
       8.69055192e-04, 1.37175629e-03, 1.20805515e-03, 7.51584483e-04,
       4.29647184e-04, 1.60895960e-04, 3.21362505e-05, 1.36197327e-03,
       1.26077700e-03, 1.20705997e-03, 1.27223099e-03, 7.12671986e-04,
       3.33021446e-04, 3.64633084e-04, 1.95749723e-03, 1.95067581e-03,
       1.44028375e-03, 1.17967762e-03, 1.81947276e-03, 1.62744094e-03,
       2.15750018e-04, 6.63668581e-05, 2.05323408e-04, 2.20936958e-04,
       2.54288070e-05, 4.96989328e-05, 3.43953698e-04, 9.30949409e-01,
       4.94792329e-03])

In [30]:
# Visualize the Random Forest model
# (Need to reduce max_depth of rf_model in order to create useful visualization)

#fig = plt.figure(figsize=(15, 10))
#plot_tree(rf_model.estimators_[0], 
#          filled=True, impurity=True, 
#          rounded=True)